In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/CLASE_47/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
nltk=3.5 already installed
unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed



<img src='../../../common/logo_DH.png' align='left' width=35%/>





<a id="section_clasification"></a>

# Clasificación de textos





<a id="section_toc"></a> 

## Tabla de Contenidos


[1- Dataset](#dataset)

[2- Limpieza](#limpieza)

[3- Vectorización y modelado](#vectorizacion)

[4- Repaso del clasificador Naive Bayes](#NBC)

[5- Implementaciones del modelo](#implementacion)




In [2]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
import numpy as np
import pickle


<a id="dataset"></a>

### El dataset

[Volver al índice](#section_toc)



Trabajaremos ahora con un dataset que contiene comentarios sobre películas hechos por usuarios del sitio IMDB.
Además de los comentarios, los usuarios dejan un puntaje de las películas, de modo que podemos saber si los comentarios son positivos o negativos de acuerdo a dicho puntaje. A los problemas de clasificación que involucran detectar una valoración positiva o negativa en un texto se los conoce como problemas de "sentiment analysis". Aquí lo abordaremos como un problema de aprendizaje supervisado, dado que tenemos las etiquetas de la valoración de cada comentario.

El dataset completo se encuentra aquí: http://ai.stanford.edu/~amaas/data/sentiment/.

Utilizaremos una versión subsampleada del mismo como ejemplo para usar menos memoria y correr más rápido los modelos.

La versión acotada se encuentra en el archivo movie_reviews.obj que podemos leer usando la librería pickle.
El data set está guardado como un objeto de la clase Bunch de sklearn, que, como veremos, se manipula igual que un diccionario de python. Dentro de este objeto veremos que tenemos los textos almacenados en el atributo "data" y las etiquetas en el atributo "target".

In [3]:

filehandler = open('../Data/movie_reviews.obj', 'rb') 
reviews = pickle.load(filehandler)
filehandler.close()

texts=reviews.data;
targets=reviews.target;

print('reviews type:',type(reviews))
print('targets type:',type(targets))
print('texts type:',type(texts))


reviews type: <class 'sklearn.utils.Bunch'>
targets type: <class 'numpy.ndarray'>
texts type: <class 'list'>


In [4]:
# Veamos como ejemplo el primer comentario
texts[0]

b"Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty."

#### Nota aparte sobre el formato:

En el ejemplo de arriba la 'b' antes del texto indica que los datos están codificados en bytes. Nosotros vemos el texto como una variable de tipo string, es decir una secuencia de caracteres, pero cada caracter ha sido mapeado a un número (un byte) mediante algún enconding (utf-8 por ejemplo). Es decir que si miramos el contenido de la variable como una lista, veremos los bytes. Al usar print() python decodifica los bytes a caracteres.

In [5]:
print(type(texts[0]))
print(list(texts[0][:15]))
print(texts[0][:15])

<class 'bytes'>
[90, 101, 114, 111, 32, 68, 97, 121, 32, 108, 101, 97, 100, 115, 32]
b'Zero Day leads '




<a id="limpieza"></a>

#### Limpieza

[Volver al índice](#section_toc)


Vemos en el ejemplo de arriba que los comentarios tienen comandos html como $\text{<br />}$ (break line). Vamos a removerlos antes de hacer nuestro análisis. Podríamos también llevar todo a minúsculas y remover signos de puntuación, pero como las herramientas como CountVectorizer y TfidfVectorizer ya tienen estos pasos incorporados no es necesario.

In [6]:
texts = [doc.replace(b"<br />", b" ") for doc in texts];
texts[0]

b"Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.  It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see.   Flawed but honest with a terrible honesty."



<a id="vectorizacion"></a>

#### Vectorización y Modelado

[Volver al índice](#section_toc)


Vamos a vectorizar el corpus primero con CountVectorizer y luego con TfidfVectorizer. Luego entrenaremos un clasificador de tipo Naive Bayes y evaluaremos la performance  usando distintas estrategias para reducir la dimensionalidad del dataset: remover stopwords de una lista, filtrar palabras que aparecen en muy pocos documentos.

Antes de empezar, separamos el dataset haciendo un train test split. Usaremos el training set para seleccionar modelos mediante cross validation y luego reportaremos la performance del modelo elegido en el test set.

In [7]:

train,test,y_train,y_test=train_test_split(texts,targets); # Por defecto el split se hace de manera estratificada: preservando las proporciones de los targets en train y test

print('train size:',len(train))
print('test size:',len(test))
print('proporcion de positivos en train:',y_train.mean())
print('proporcion de positivos en test:',y_test.mean())


train size: 18750
test size: 6250
proporcion de positivos en train: 0.5045866666666666
proporcion de positivos en test: 0.48624


#### Vectorización

Usamos CountVectorizer para lleva nuestro corpus a una matriz de documentos y términos.

In [8]:

vectorizer=CountVectorizer(strip_accents='unicode'); # Si bien los comentarios están en inglés, hay palabras con acentos por error de tipeo o por ser palabras de otro idioma. Los removemos.
X_train=vectorizer.fit_transform(train);

print('Dimensionalidad del dataset:')
X_train.shape

Dimensionalidad del dataset:


(18750, 66297)

Vemos que el corpus está compuesto por más de 60 mil términos. Veamos los primeros, los últimos y algunos sampleados equiespaciados (cada 700 palabras) dentro del vocabulario:

In [9]:
print('Los primeros términos:')
print(vectorizer.get_feature_names()[:100])

print('\n Los últimos términos:')
print(vectorizer.get_feature_names()[-100:])

print('\n Términos equiespaciados dentro del vocabulario:')
print(vectorizer.get_feature_names()[::700])


Los primeros términos:
['00', '000', '0000000000001', '00001', '00015', '000s', '001', '006', '007', '0080', '0083', '0093638', '00am', '00pm', '00s', '01', '01pm', '02', '03', '04', '041', '05', '06', '06th', '07', '08', '089', '08th', '09', '0f', '0r', '10', '100', '1000', '1000000', '1000s', '1001', '100b', '100k', '100min', '100mph', '100s', '100th', '100x', '100yards', '101', '101st', '102', '102nd', '103', '104', '1040', '1040a', '1040s', '105', '105lbs', '106', '107', '108', '109', '10am', '10lines', '10mil', '10minutes', '10p', '10pm', '10s', '10th', '10x', '10yr', '11', '110', '1100', '11001001', '1100ad', '111', '112', '1138', '114', '1146', '115', '116', '117', '11m', '11th', '12', '120', '1200', '1201', '1202', '123', '12383499143743701', '125', '127', '128', '12a', '12hr', '12m', '12mm', '12s']

 Los últimos términos:
['ziyi', 'zizek', 'zizekian', 'zizte', 'zmed', 'zmeu', 'znaimer', 'zo', 'zobie', 'zod', 'zodiac', 'zodsworth', 'zoe', 'zoey', 'zohar', 'zoheb', 'zoimbies', '



<a id="NBC"></a>

#### Breve repaso de Multinomial Naive Bayes 

[Volver al índice](#section_toc)

Implementaremos un clasificador multinomial naive bayes.

El problema de clasificación consiste en estimar una probabilidad de que la clase (Y) tome un cierto valor (y) 
dado que las features (X) tomaron los valores observados ($x_{obs}$): $P(Y=y|X=x_{obs})$.

El teorema de Bayes permite descomponer esta probabilidad condicional del siguiente modo:


\begin{equation}
P(Y=y|X=x_{obs})=\frac{P(X=x_{obs}|Y=y)P(Y=y)}{P(X=x_{obs})}
\end{equation}

La variable de clase Y puede tomar uno entre varios valores y a la hora de hacer una predicción elegimos el más probable:

\begin{equation}
\hat{y}=argmax_y  P(Y=y|X=x_{obs})
\end{equation}

Esta elección es independiente del valor de $P(X=x_{obs})$ que no depende de y, de modo que sólo es necesario calcular

\begin{equation}
P(Y=y|X=x_{obs})\sim P(X=x_{obs}|Y=y)P(Y=y)
\end{equation}

P(Y=y) se conoce como prior y es nuestra estimación de cuán probable es que una observación pertenezca a una clase independiendemente del valor de las features. Puede asumirse un prior uniforme (todas las clases son equiprobables) o podemos estimarlo de los datos: cuán representada está cada clase.

El otro factor es la verosimilitud o likelihood $P(X=x_{obs}|Y=y)$ y depende del modelo que vayamos a usar. La palabra Naive dentro del modelo implica que podemos asumir independencia entre las features de modo que podemos factorizar esta probabilidad como

\begin{equation}
P(Y=y|X=x_{obs})=\prod_i P(X_i=x_i|Y=y)
\end{equation}

en donde $X_i$ es la feature i-ésima.

En el caso de clasificación de texto, los valores de las features ($x_i$) son frecuencias de aparición de palabras en los documentos, por lo que es razonable usar un modelo multinomial (Multinomial Naive Bayes) en donde cada uno de esos factores lo podemos calcular como

\begin{equation}
P(X_i=x_i|Y=y)={\theta_{yi}}^{x_i}
\end{equation}

en donde $\theta_{yi}$ es la probabilidad de que aparezca el término i-ésimo en un documento de clase $y$ y se puede estimar como

\begin{equation}
\hat{\theta}_{yi}=\frac{N_{yi}+\alpha}{N_y + \alpha n}
\end{equation}

en donde $N_{yi}$ es el número total de veces que se observó el término i-ésimo en documentos de clase $y$, n es el número de términos, $N_y$ es el número total de ocurrencias de todos los términos dentro de la clase $y$.

$\alpha$ es un hiperparámetro del modelo que implica un suavizado de la distribución de probabilidades. Cuando $\alpha >0$, a los términos que aparecen cero veces en un documento se les asigna una pequeña probabilidad.


https://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes

<a id="implementacions"></a>

#### Implementación del modelo


[Volver al índice](#section_toc)


Vamos a implementar el modelo, optimizando el hiperparámetro $\alpha$ mediante una grid search cross validation

In [10]:

skf=StratifiedKFold(n_splits=3,random_state=0,shuffle=True)

param_grid={'alpha':np.arange(0.05,1,0.05)};
grid = GridSearchCV(MultinomialNB(), param_grid, cv=skf,verbose=1);
grid.fit(X_train, y_train);
print("Best cross-validation score: {:.4f}".format(grid.best_score_));
print("Best parameters: ", grid.best_params_);

Fitting 3 folds for each of 19 candidates, totalling 57 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best cross-validation score: 0.8463
Best parameters:  {'alpha': 0.8500000000000001}


[Parallel(n_jobs=1)]: Done  57 out of  57 | elapsed:    1.4s finished


#### Tf-idf

Comparemos la performance con la obtenida usando la representacion tf-idf

In [11]:

tfidf=TfidfTransformer();
X_tfidf=tfidf.fit_transform(X_train);

param_grid={'alpha':np.arange(0.05,1,0.05)};
grid = GridSearchCV(MultinomialNB(), param_grid, cv=skf,verbose=1);
grid.fit(X_tfidf, y_train);
print("Best cross-validation score: {:.4f}".format(grid.best_score_));
print("Best parameters: ", grid.best_params_);

Fitting 3 folds for each of 19 candidates, totalling 57 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best cross-validation score: 0.8634
Best parameters:  {'alpha': 0.55}


[Parallel(n_jobs=1)]: Done  57 out of  57 | elapsed:    1.3s finished


Vemos una performance levemente mejor con Tf-idf. Veamos si podemos mejorar esta performace bajando la dimensionalidad de los datos. Empecemos por remover stopwords. 

In [12]:

from nltk.corpus import stopwords 
stop_words=stopwords.words('english');

vectorizer=TfidfVectorizer(stop_words=stop_words,strip_accents='unicode');

X_tfidf_reduced=vectorizer.fit_transform(train);

grid.fit(X_tfidf_reduced, y_train);
print("Best cross-validation score: {:.4f}".format(grid.best_score_));
print("Best parameters: ", grid.best_params_);


Fitting 3 folds for each of 19 candidates, totalling 57 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best cross-validation score: 0.8655
Best parameters:  {'alpha': 0.9000000000000001}


[Parallel(n_jobs=1)]: Done  57 out of  57 | elapsed:    0.9s finished


Vemos una leve mejoría, pero veamos cuánto hemos reducido efectivamente la dimensionalidad del dataset:

In [13]:
print('Con stopwords:',X_train.shape)
print('Sin stopwords:',X_tfidf_reduced.shape)

Con stopwords: (18750, 66297)
Sin stopwords: (18750, 66152)


Veamos qué pasa si removemos términos que aparecen en muchos documentos: stopwords específicas de este corpus. El parámetro max_df remueve términos que aparecen en más que x documentos. Probemos removiendo palabras que aparecen en más de 100 documentos.

In [14]:
vectorizer=TfidfVectorizer(stop_words=stop_words,strip_accents='unicode',max_df=100);

X_tfidf_reduced=vectorizer.fit_transform(train);

print('Dimensionalidad:',X_tfidf_reduced.shape)

grid.fit(X_tfidf_reduced, y_train);
print("Best cross-validation score: {:.4f}".format(grid.best_score_));
print("Best parameters: ", grid.best_params_);


Dimensionalidad: (18750, 63257)
Fitting 3 folds for each of 19 candidates, totalling 57 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best cross-validation score: 0.7925
Best parameters:  {'alpha': 0.55}


[Parallel(n_jobs=1)]: Done  57 out of  57 | elapsed:    0.8s finished


Vemos que el modelo empeoró. Si juegan un poco con este parámetro verán que luego de remover stopwords, no hay muchos términos que aparezcan en un porcentaje alto documentos del corpus. Esto se debe a que los comentarios sobre películas son bastante cortos y muchas palabras refieren específicamente al film.

#### Bigramas

Probemos ahora incluyendo bigramas en el vocabulario. Es decir, considerar como términos a pares de palabras consecutivas en el texto. Tanto CountVectorizer como TfidfVectorizer tienen el parámetro ngram_range.

De la documentación:

<b> ngram_range</b>: tuple (min_n, max_n), default=(1, 1)
The lower and upper boundary of the range of n-values for different n-grams to be extracted. All values of n such that min_n <= n <= max_n will be used. For example an ngram_range of (1, 1) means only unigrams, (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams.

In [15]:
# Incluimos bigramas

vectorizer=TfidfVectorizer(stop_words=stop_words,strip_accents='unicode',ngram_range=(1,2));
X_tfidf_bigrams=vectorizer.fit_transform(train);

print('Dimensiones de la matriz:', X_tfidf_bigrams.shape,'\n')


grid.fit(X_tfidf_bigrams, y_train);
print("Best cross-validation score: {:.4f}".format(grid.best_score_));
print("Best parameters: ", grid.best_params_);



Dimensiones de la matriz: (18750, 1464314) 

Fitting 3 folds for each of 19 candidates, totalling 57 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best cross-validation score: 0.8843
Best parameters:  {'alpha': 0.15000000000000002}


[Parallel(n_jobs=1)]: Done  57 out of  57 | elapsed:    9.1s finished


In [16]:
# Performance de nuestro modelo en el test set:
from sklearn.metrics import accuracy_score

X_test=vectorizer.transform(test);
model=grid.best_estimator_;
y_pred=model.predict(X_test);

print('Test accuracy:',accuracy_score(y_test,y_pred))


Test accuracy: 0.8936


Veamos qué predicción hace nuestro modelo sobre comentarios escritos por nosotros:

In [17]:
# 
classes={0:'negativo',1:'positivo'};

texto_prueba=['this movie is terrible. I have never seen anything this bad in my life.']
texto_vec=vectorizer.transform(texto_prueba);
clase=model.predict(texto_vec);

print('Comentario:',texto_prueba,'\n')
print('Calificación:',classes[clase[0]],' con proba:',model.predict_proba(texto_vec).max())

texto_prueba=['I highly recomend this film to anyone who likes documentaries. The director succeded in portraiyng the soul of the main character.']
texto_vec=vectorizer.transform(texto_prueba);
clase=model.predict(texto_vec);
print('\nComentario:',texto_prueba,'\n')
print('Calificación:',classes[clase[0]],' con proba:',model.predict_proba(texto_vec).max())


Comentario: ['this movie is terrible. I have never seen anything this bad in my life.'] 

Calificación: negativo  con proba: 0.9183280839667625

Comentario: ['I highly recomend this film to anyone who likes documentaries. The director succeded in portraiyng the soul of the main character.'] 

Calificación: positivo  con proba: 0.8044476111771026


#### Comentario sobre la optimización del modelo

Hemos estado optimizando el hiperparámetro $\alpha$ y moviendo a mano otros hiperparámetros que intervienen en la vectorización: min_df, max_df, inclusión o no de stopwords.

Para ser más precisos deberíamos armar un pipeline que incluya la verctorizacion y el clasificador para optimizar sobre todos los hiperparámetros a la vez. Obviamente esto tiene un mayor costo de cómputo, pero dejamos un código aquí abajo como referencia en donde se optimiza alpha y min_df



In [18]:
from sklearn.pipeline import Pipeline

vectorizer=TfidfVectorizer(stop_words=stop_words,strip_accents='unicode');

model=Pipeline([('vect',vectorizer),('classifier',MultinomialNB())])

params={'classifier__alpha':[0.1,0.5,1],'vect__min_df':[1,3]};

GS_CV=GridSearchCV(model,params,cv=skf,verbose=1,n_jobs=-1);

GS_CV.fit(train, y_train);

print('best score:',GS_CV.best_score_)
print('best params:',GS_CV.best_params_)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   32.4s finished


best score: 0.8657066666666666
best params: {'classifier__alpha': 1, 'vect__min_df': 1}


### Más datasets

Para practicar pueden buscar más datasets en los siguientes sitios:

https://lionbridge.ai/datasets/14-best-text-classification-datasets-for-machine-learning/

https://lionbridge.ai/datasets/15-free-sentiment-analysis-datasets-for-machine-learning/

NLTK también provee datasets que pueden descargar desde la linea de comandos y manipularlos fácilmente con funciones de la librería. 

Referencia: https://www.nltk.org/book/ch02.html
